In [59]:
#全データに対応できる汎用的なモデルを作成する
#SIRMsファジィ推論モデルを作成する
import numpy as np
import pandas as pd
import random
import math
import sys
import time

In [60]:
#各種データを読み込む
original_data = pd.read_csv('winequality-white-test.csv')

In [61]:
#次に各種データに対して正規化を行っていく
#ただし、例外のカラムを設けておく
#正規化をあらかじめ行いたいカラムについてはその例外にカラム名を付与しておく

#まず例外カラム名の制定
#各種データごとに確認・変更してほしい箇所
##########################################################################################################################
##########################################################################################################################
exception_columns = []
for clm in original_data.columns:
    if clm not in exception_columns:
        original_data[clm] = (original_data[clm] - original_data[clm].min()) / (original_data[clm].max() - original_data[clm].min())
##########################################################################################################################
##########################################################################################################################

In [62]:
#ここで各種データをnumpy配列に変換しておく
np_original_data = np.array(original_data)

In [63]:
#次に教師データとテストデータに分割する
#ここでは教師データ:テストデータを5:5とする

#教師データ、テストデータの割合の設定
#各種データごとに確認・変更してほしい箇所
##########################################################################################################################
##########################################################################################################################
TEACHER_SIZE_RATIO = 5
TEST_SIZE_RATIO = 5
##########################################################################################################################
##########################################################################################################################

#上で定めた割合に基づき、教師データ数とテストデータ数を求める
TEACHER_SIZE = ((int)(len(np_original_data) * ((TEACHER_SIZE_RATIO) / (TEACHER_SIZE_RATIO + TEST_SIZE_RATIO)))) + 1
TEST_SIZE = len(np_original_data) - TEACHER_SIZE

In [64]:
#次に教師データとテストデータに分割する
teacher_data = np_original_data[:TEACHER_SIZE,:]
test_data = np_original_data[TEACHER_SIZE:,:]

In [65]:
#元データの入力データと出力データの合計の属性数を求める
NUMBER_OF_INPUT_OUTPUT= len(teacher_data[0,:])
NUMBER_OF_INPUT = NUMBER_OF_INPUT_OUTPUT - 1

In [66]:
#説明変数と被説明変数とに分割する
#被説明変数は最後の要素のみとする
x_teacher_data = teacher_data[:,:NUMBER_OF_INPUT]
y_teacher_data = teacher_data[:,NUMBER_OF_INPUT]
x_test_data = test_data[:,:NUMBER_OF_INPUT]
y_test_data = test_data[:,NUMBER_OF_INPUT]

In [67]:
#次にファジィ分割数と入力データの属性数からファジィルールの総数を求める
#各種データごとに確認・変更してほしい箇所
##########################################################################################################################
##########################################################################################################################
NUMBER_OF_FUZZY_PARTITION = 3
#SIRMsファジィ推論モデルはTSKファジィ推論を入力変数の数を行い出力結果を加重平均取る
NUMBER_OF_INPUT_OF_FUZZY_RULE = 1
NUMBER_OF_FUZZY_RULE = NUMBER_OF_FUZZY_PARTITION ** NUMBER_OF_INPUT_OF_FUZZY_RULE
##########################################################################################################################
##########################################################################################################################

In [68]:
#前件部の初期値と後件部の初期値を設定する
#また、SIRMsとモジュラー限定だが各TSKファジィ推論器ごとの重みも設定する
#前件部の中心と幅は、ルール数×各ルールにおける入力変数の数
antecedent_center = np.empty((NUMBER_OF_INPUT, NUMBER_OF_FUZZY_RULE))
antecedent_broad = np.empty((NUMBER_OF_INPUT, NUMBER_OF_FUZZY_RULE))
consequent = np.empty((NUMBER_OF_INPUT, NUMBER_OF_FUZZY_RULE))
weight_of_tsk_fuzzy = np.empty(NUMBER_OF_INPUT)

#各種データごとに確認・変更してほしい箇所
##########################################################################################################################
##########################################################################################################################
#後件部の値の初期設定を行う
for i in range(NUMBER_OF_INPUT):
    for j in range(NUMBER_OF_FUZZY_RULE):
        consequent[i,j] = 0.5
#各TSKファジィ推論器における重みを求める
for i in range(NUMBER_OF_INPUT):
    weight_of_tsk_fuzzy[i] = 0.2
#前件部を初期設定する
for i in range(NUMBER_OF_INPUT):
    for j in range(NUMBER_OF_FUZZY_RULE):
        if j == 0:
            antecedent_center[i,j] = 0
            antecedent_broad[i,j] = 1
        elif j == 1:
            antecedent_center[i,j] = 0.5
            antecedent_broad[i,j] = 0.5
        else:
            antecedent_center[i,j] = 1
            antecedent_broad[i,j] = 1
##########################################################################################################################
##########################################################################################################################

In [ ]:
%%time

#学習を行う
#前件部の学習はここでは考えないことにする
#学習回数を設定する
TRAIN_TIME = 1000

mean_squared_error = 0

#学習係数を設定する
LEARNING_CONSEQUENT = 0.001
LEARNING_WEIGHT = 0.001

#各学習に関して
for time in range(TRAIN_TIME):
    #SIRMsファジィ推論の出力値を設定しておく
    output = np.empty(TEACHER_SIZE)
    #各教師データに対して
    for i in range(TEACHER_SIZE):
        #SIRMs内の各TSKファジィ推論の推論結果を格納する変数を設定しておく
        tsk_output = np.empty(NUMBER_OF_INPUT)
        #各TSKファジィ推論内の各ルールに対する適合度を求める
        adaptability = np.empty((NUMBER_OF_INPUT, NUMBER_OF_FUZZY_RULE))
        #SIRMs内の各TSKファジィ推論に対して
        #ただし、TSKファジィ推論器の数は各教師データの入力属性数に等しい
        for j in range(NUMBER_OF_INPUT):
            #各ファジィルールに対して
            for k in range(NUMBER_OF_FUZZY_RULE):
                if(x_teacher_data[i,j] >= antecedent_center[j,k] - antecedent_broad[j,k]) and (x_teacher_data[i,j] <= antecedent_center[j,k]):
                    adaptability[j,k] = (x_teacher_data[i,j] - (antecedent_center[j,k] - antecedent_broad[j,k])) / antecedent_broad[j,k]
                elif(x_teacher_data[i,j] > antecedent_center[j,k]) and (x_teacher_data[i,j] <= antecedent_center[j,k] + antecedent_broad[j,k]):
                    adaptability[j,k] = -(x_teacher_data[i,j] - (antecedent_center[j,k] + antecedent_broad[j,k])) / antecedent_broad[j,k]
            #各TSKファジィ推論器ごとの出力値を求める
            tsk_output[j] = np.sum(np.dot(adaptability[j], consequent[j])) / np.sum(adaptability[j])
        #SIRMsファジィ推論における出力値を求める
        output[i] = np.sum(np.dot(weight_of_tsk_fuzzy, tsk_output))
        if time == TRAIN_TIME - 1:
            print(output[i], y_teacher_data[i])
            mean_squared_error = mean_squared_error + (output[i] - y_teacher_data[i])**2
            if i == TEACHER_SIZE-1:
                mean_squared_error = mean_squared_error / TEACHER_SIZE
                print("平均二条誤差")
                print(mean_squared_error)
        #各種パラメータの更新を行う
        #ここでは後件部の値と各TSKファジィ推論の重みを更新する
        #まず後件部の値の更新を行う
        for j in range(NUMBER_OF_INPUT):
            for k in range(NUMBER_OF_FUZZY_RULE):
                consequent[j,k] = consequent[j,k] + (LEARNING_CONSEQUENT * (y_teacher_data[i] - output[i]) * weight_of_tsk_fuzzy[j] * (adaptability[j,k] / np.sum(adaptability[j])))
        #次に結合重みを更新する
        for j in range(NUMBER_OF_INPUT):
            weight_of_tsk_fuzzy[j] = weight_of_tsk_fuzzy[j] + (LEARNING_WEIGHT * (y_teacher_data[i] - output[i]) * tsk_output[j])

In [ ]:
#テストデータを用いて予測結果を求める
#SIRMsファジィ推論モデルの出力値を設定しておく
test_output = np.empty(TEST_SIZE)

test_mean_squared_error = 0

#各テストデータに対して
for i in range(TEST_SIZE):
    #SIRMs内の各TSKファジィ推論の推論結果を格納する変数を設定しておく
    test_tsk_output = np.empty(NUMBER_OF_INPUT)
    #各TSKファジィ推論内の各ルールに対する適合度を求める
    test_adaptability = np.empty((NUMBER_OF_INPUT, NUMBER_OF_FUZZY_RULE))
    #SIRMs内の各TSKファジィ推論器に対して
    for j in range(NUMBER_OF_INPUT):
        for k in range(NUMBER_OF_FUZZY_RULE):
            if(x_test_data[i,j] >= antecedent_center[j,k] - antecedent_broad[j,k]) and (x_test_data[i,j] <= antecedent_center[j,k]):
                test_adaptability[j,k] = (x_test_data[i,j] - (antecedent_center[j,k] - antecedent_broad[j,k])) / antecedent_broad[j,k]
            elif(x_test_data[i,j] > antecedent_center[j,k]) and (x_test_data[i,j] <= antecedent_center[j,k] + antecedent_broad[j,k]):
                test_adaptability[j,k] = -(x_test_data[i,j] - (antecedent_center[j,k] + antecedent_broad[j,k])) / antecedent_broad[j,k]
        #各TSKファジィ推論器ごとの出力値を求める
        test_tsk_output[j] = np.sum(np.dot(test_adaptability[j], consequent[j])) / np.sum(adaptability[j])
    #SIRMsファジィ推論における出力値を求める
    test_output[i] = np.sum(np.dot(weight_of_tsk_fuzzy, test_tsk_output))
    print(output[i], y_test_data[i])
    test_mean_squared_error = test_mean_squared_error + (output[i] - y_test_data[i])**2
    if i == TEST_SIZE-1:
        test_mean_squared_error = test_mean_squared_error / TEST_SIZE
        print("平均二条誤差")
        print(test_mean_squared_error)